## UPAC96 Self Trigger

This notebook demostrates how to use the self trigger of the UPAC96 to capture events. 

### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `UPAC96`
    + Firmware: `v911` - `v911`


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

from naludaq.board import Board, startup_board
from naludaq.daq import DebugDaq
from naludaq.parsers import Upac96Parser
from naludaq.controllers import get_board_controller, get_dac_controller, get_trigger_controller

## Board Connection and Persistent Objects

In [ ]:
board = Board('upac96')
board.load_registers()
board.get_ftdi_connection(serial_number='FTCKIDF5', baud=2000000)
startup_board(board)

### Channel Trigger Mask

In [ ]:
# Set Channels to self trigger on
trig_ctrl = get_trigger_controller(board)

# Triggering on Channel 0 and 2 is set to enabled 
trig_ctrl.set_channel_trigger_mask([0, 2]) # Format: [0, 1, 2, 5, 7, 23 ...]


### Trigger Thresholds

In [ ]:
trig_ctrl = get_trigger_controller(board)

# Set trigger values to desired values, then set the
# trigger controller trigger_values, default trigger value is 0
trigger_values = [0] * 96 # Initialize a list of trigger values, 
trigger_values[2] = 1800 # Sets Channel 2 to a trigger value of 1800
trig_ctrl.trigger_values = trigger_values

### Trigger Edges

In [ ]:
# Channel edges is a list of 96 bools, for the 96 channels, default is false
# False is falling edge, and True is rising edge
trig_ctrl = get_trigger_controller(board)
channel_edges = [False] * 96 # Initialize a list of edge values for all channels
channel_edges[10] = True # Set Channel 10 to trigger on rising edge
trig_ctrl.set_trigger_edge(channel_edges)

### Coincidence Trigger

In [ ]:
# Input takes a dict of chips you would like to set, mapping the chip num
# to a bool value stating if whether to set a chip to coincidence
# trigger mode, by default coicidence trigger is disabled
trig_ctrl = get_trigger_controller(board)
# Set Channel 0 to regular trigger mode, triggering when any enabled channels meets their trigger condition
# Set Channel 1 & 4 to coincidence trigger mode, triggering when all enabled channels meets their trigger condition
coincidences = {0: False, 1: True, 4: True}
trig_ctrl.set_coincidence_trigger_enabled(coincidences)

In [ ]:
# If you don't have a function generator hooked up,
# you can sweep over the dac values to trigger
trig_ctrl = get_trigger_controller(board)
bc = get_board_controller(board)
dc = get_dac_controller(board)

ddaq = DebugDaq(board, store_raw_data=True)
ddaq.start_capture()
time.sleep(0.1)
dc.set_dacs(1800, channels=[0])
time.sleep(0.2)
# Set continuous mode if you want test triggering more than once
# bc.set_continuous_mode(True)
bc.start_readout("self")
try:
    for dac in range(2000, 2400, 100):
        dc.set_dacs(dac, channels=[0])
        time.sleep(2)
        if len(ddaq.output_buffer) > 0:
            print(f"Triggered on DAC value: {dac}")
            break
except KeyboardInterrupt:
    pass
finally:
    bc.stop_readout()
    ddaq.stop_capture()
    print(f"Output Buffer len: {len(ddaq.output_buffer)}")

### For testing with a function generator

A square wave is recommended if testing with a function generator,
Starting input could be, 400 mVpp, 2 MHz

In [ ]:
parser = Upac96Parser(board.params)
ddaq = DebugDaq(board, store_raw_data=True, custom_parser = parser)
ddaq.start_capture()
time.sleep(0.1)
dc.set_dacs(2300, channels=[0, 32]) # channels take input 0 for channels 0-15, 16 for 16-31, ... 
time.sleep(0.2)
# Set continuous mode if you want test triggering more than once
# bc.set_continuous_mode(True)
bc.start_readout("self")
try:
    while len(ddaq.output_buffer) <= 0:
        time.sleep(2)
except KeyboardInterrupt:
    pass
finally:
    bc.stop_readout()
    ddaq.stop_capture()
    print(f"Output Buffer len: {len(ddaq.output_buffer)}")

### Plot Data

In [ ]:
%matplotlib widget

# Setup graph style
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'font.family': 'monospace'})
data = ddaq.output_buffer[0]['data']
fig = plt.figure(figsize=(8,6), constrained_layout=True)
plt.xlabel('Time')
plt.ylabel('ADC Counts')
plt.title('UPAC96 Capture')
for channel in [*range(0,16)]:
    ch_data = data[channel]
    ax = plt.plot(ch_data, '.-', label=f"Channel {channel}")
plt.legend()
plt.show()

In [ ]:
board.disconnect()